In [1]:
import tweepy
from kafka import KafkaProducer
from json import dumps
import time
import json

In [2]:
#Authenticate on Twitter

consumer_key = "dmn51aowr3bYzIWz3ELgARN5x"
consumer_secret = "Q6vHf7FNj1yDvX2vggN6kKLgiDP7t7HL5DNpNvK0akyjzfxTFt"
access_token = "164266780-hcTZEl441WUst50TpiRIZtAd3IHmvPDX5tQUDjqx"
access_token_secret = "AIoEwy9AL52AuHRAKesBph3arofQ4AdLwqYRl8tFnehxd"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [3]:
#Create Kafka Producer

ip = "35.228.250.247"
producer = KafkaProducer(bootstrap_servers=[ip+':9092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

In [4]:
#Retrieve TopN Hashtags on the given location

location = 2487956 #san francisco
n = 2 #top n parameter
top_n = [x.get("name") for x in api.trends_place(location)[0].get('trends')[0:n]]
print(top_n)

['#DemDebate', 'World Series']


In [5]:
#Define Listener

class ParallelListener(tweepy.StreamListener):
    def __init__(self,time_limit=60):
        self.start_time = time.time()
        self.limit = time_limit
        super(ParallelListener, self).__init__()

    def on_status(self, status):
        if (time.time() - self.start_time) < self.limit:
            data = {}
            data['date'] = str(status.created_at)
            data['tweet'] = status.text
            json_data = json.dumps(data)
            producer.send("test",json_data)
            print(json_data)
            return True
        else:
            print("Stopping stream..")
            return False

#Launch all listener together

listener = ParallelListener(time_limit=1000)
myStream = tweepy.Stream(auth = api.auth, listener=listener)
myStream.filter(track=top_n, is_async=False, languages=["en"])

{"date": "2019-10-16 10:28:17", "tweet": "RT @AndrewYang: \"We need to decriminalize opiates for personal use, we have to let the country know this is not a personal failing, this wa\u2026"}
{"date": "2019-10-16 10:28:17", "tweet": "RT @51stDC: We have a historic opportunity to raise awareness about #DCStatehood and to win a World Series! #StayInTheFight"}
{"date": "2019-10-16 10:28:18", "tweet": "RT @JayInslee: Not one single question about the climate crisis. \nNot one single question about the climate crisis. \nNot one single questio\u2026"}
{"date": "2019-10-16 10:28:18", "tweet": "@suzyonthecape Posted... \nhttps://t.co/6M2rVMjRCc"}
{"date": "2019-10-16 10:28:18", "tweet": "RT @JayInslee: Not one single question about the climate crisis. \nNot one single question about the climate crisis. \nNot one single questio\u2026"}
{"date": "2019-10-16 10:28:18", "tweet": "RT @JayInslee: Not one single question about the climate crisis. \nNot one single question about the climate crisis. \nN

{"date": "2019-10-16 10:28:25", "tweet": "RT @BetoORourke: Freedom is being able to go to school without being afraid of getting shot. https://t.co/MMB3VtBx8b"}


KeyboardInterrupt: 

In [7]:
#Define Listener

class PipelineListener(tweepy.StreamListener):
    def __init__(self,tag,time_limit=60):
        self.start_time = time.time()
        self.limit = time_limit
        self.tag=tag
        super(PipelineListener, self).__init__()

    def on_status(self, status):
        if (time.time() - self.start_time) < self.limit:
            data = {}
            data['tag'] = self.tag
            data['date'] = str(status.created_at)
            data['tweet'] = status.text
            json_data = json.dumps(data)
            producer.send("test",json_data)
            print(json_data)
            return True
        else:
            print("Stopping stream..")
            return False

#Launch listener in series

time_limit = 40

for _ in range(0,5):
    listeners = []
    streams = []
    for i in range(0,n):
        print(top_n[i])
        listeners.append(PipelineListener(top_n[i],time_limit=time_limit))
        streams.append(tweepy.Stream(auth = api.auth, listener=listeners[i]))
        streams[i].filter(track=[top_n[i]], is_async=False, languages=["en"])
        streams[i].disconnect()

#DemDebate
{"tag": "#DemDebate", "date": "2019-10-16 10:29:54", "tweet": "RT @andyash82362457: Watching the #DemDebate tonight, I am seeing more of what @PeteButtigieg said weeks ago about EW\u2019s \u201cmy way or the high\u2026"}
{"tag": "#DemDebate", "date": "2019-10-16 10:29:54", "tweet": "RT @LordMaountain: @RoKhanna @secularcitizen2 @BernieSanders #BernieSanders has always been concise, precise and consistent on how he prote\u2026"}
{"tag": "#DemDebate", "date": "2019-10-16 10:29:55", "tweet": "RT @mariashriver: Bravo @corybooker who says we have one shot at making @realdonaldtrump a one term President and how we talk to each other\u2026"}
{"tag": "#DemDebate", "date": "2019-10-16 10:29:55", "tweet": "RT @annavivictor: So sad.. https://t.co/ptpnQ7xlDF"}
{"tag": "#DemDebate", "date": "2019-10-16 10:29:55", "tweet": "RT @JoeBiden: \u201cWe have an erratic, crazy President who knows not a damn thing about foreign policy.\u201d\n\n#DemDebate"}
{"tag": "#DemDebate", "date": "2019-10-1

KeyboardInterrupt: 